## Ofsted Inspection Reports Scrape Tool 


<b>Summary:</b><br>
<span style="font-size:10pt">Scrapes local authorities 'Childrens Services Ofsted Inspection' reports from https://reports.ofsted.gov.uk/ in pdf format. The Children's Services Inspection reports are downloaded locally as their original pdf, organised into folders by local authority name and their URN. The LA naming is cleaned to better reflect a more standard/onward process use of LA naming - e.g. 'Nottingham City Council' becomes 'Nottingham' and 'Barnsley Metropolitan Borough Council' becomes 'Barnsley'. </span><br>
    <b>Folder naming convention:</b><br>
        <span style="font-size:10pt">\provider_urn+local_authority_name(lowercase)...pdf inspection files</span><br>
<br>
<b>N.B/Pre-requisites:</b><br>
<span style="font-size:10pt">Relies on Ofsted's continued use of nonvisual css element descriptors on the web site. Obv not ideal to rely on anything in the web-space, but any scrape process, however robust, is undermined/dictated by subsequent page changes. The tool has avoided the use of Selenium or similar as this is more likely to be impacted by visual design changes on the page(s). Instead it relies on the underlying php search process, and associated php generated links.</span>

<b>Backlog/to-do:</b><br>

<ul style="font-size:10pt; list-style-type:disc; margin-left: 20px;">
    <li>Improve the loop around the pdf scrape process to skip over once most recent obtained.</li>
    <li>Scrape all/more granular inspection grades not just overall effectiveness.</li>
    <li>Be able to set a year threshold for reports to ignore historic (esp relevant to previous due to formatting issues on early reports.)</li>
    <li>Obtain/auto-lookup region codes so these can be added to LA summary export file with LA name.</li>
    <li>Are old LA numbers still being used? Add those next to URN.</li>
    <li>Local folder link in excel export to enable direct link from summary report to saved pdf files.</li>
    <li>...</li>
    <li>...</li>
</ul> 

In [11]:
#
# Export options

export_summary_filename = 'ofsted_tracker_update'
export_file_type         = 'csv' # Excel / csv currently supported


# scrape inspection grade/data from pdf reports
pdf_data_capture = True # True is default (scrape within pdf inspection reports)
                        # This impacts run time E.g False == ~1m20 / True == ~ 4m10
                        # False == only pdfs/list of LA's+link to most recent exported. Not grades

In [9]:
#
# Ofsted site/page admin settings

max_page_results = 200 # Set max number of search results to show on page(MUST be > total number of LA's!) 
url_stem = 'https://reports.ofsted.gov.uk/'
search_url = 'search?q=&location=&lat=&lon=&radius=&level_1_types=3&level_2_types%5B%5D=12' # On to-do list
max_page_results_url = '&rows=' + str(max_page_results) # Coerce results page to display ALL providers on single results page without next/pagination

# resultant complete url to process
url = url_stem + search_url + max_page_results_url




In [5]:
#
# Script admin settings

# Keep warnings quiet unless priority
import logging
import subprocess
logging.getLogger('org.apache.pdfbox').setLevel(logging.ERROR)
subprocess.call(['my_command'], stderr=subprocess.DEVNULL)

                    

In [6]:
# # Non-standard modules that might need installing
# !pip install PyPDF2
# !pip install tabula-py

In [7]:

import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

from requests.exceptions import RequestException

# pdf search/data extraction
import io
import tabula   
import PyPDF2   
import re       

# used in handling inspection dates
from dateutil import parser 
import datetime


In [8]:
#
# Function defs

def get_soup(url):
    """
    Given a URL, returns a BeautifulSoup object.
    Args: url (str): The URL to fetch and parse.
    Returns: BeautifulSoup: The parsed HTML content.
    """
    timeout_seconds = 10 # lets not assume the Ofsted page is up

    try:
        response = requests.get(url, timeout=timeout_seconds)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup
    except RequestException as e:
        print(f"An error occurred while fetching the URL '{url}': {e}")
        return None


def clean_provider_name(name):
    """
    Cleans the name according to the logic provided.
    Args:
        name (str): The original name to be cleaned.
    Returns:
        str: The cleaned name.
    """
    # Convert to lowercase and remove extra spaces
    name = name.lower().replace('  ', ' ')
    
    # Remove specific phrases
    name = name.replace("city of ", "").replace("metropolitan district council", "").replace("london borough of", "").replace("council of", "")
    
    # Remove undesired words and join the remaining parts
    name_parts = [part for part in name.split() if part not in ['city', 'metropolitan', 'borough', 'council', 'county', 'district', 'the']]
    return ' '.join(name_parts)



def extract_inspection_data(pdf_content):
    """
    Extracts the inspector's name, overall Ofsted grade, and inspection dates from the first page of a PDF report.

    Args:
        pdf_content (bytes): The content of the PDF file as bytes.

    Returns:
        dict: A dictionary containing the inspector's name, overall Ofsted grade, and inspection dates, or None if not found.

    Notes:
        This function extracts information from the first page of the PDF report. The inspector's name is extracted using a
        regular expression search for the string "Lead inspector:". The overall Ofsted grade is extracted from a table that
        appears on the first page of the report. The function uses the tabula library to extract the table data. The inspection
        dates are also extracted using a regular expression search for the string "Inspection dates:". The function attempts to
        parse the inspection dates into datetime objects and format them as "dd/mm/yyyy". The final output is a dictionary
        containing the extracted information or None if any of the information could not be found.

    Raises:
        ValueError: Raised when an unknown grade value is found during grade extraction.
    """

    # Create a file-like buffer for the PDF content
    with io.BytesIO(pdf_content) as buffer:
        # Read the PDF content for text extraction
        reader = PyPDF2.PdfReader(buffer)
        page = reader.pages[0]
        text = page.extract_text()

        # Find the inspector's name using a regular expression
        match = re.search(r"Lead inspector:\s*(.+)", text)
        if match:
            inspector_name = match.group(1)
            
            inspector_name = inspector_name.split(',')[0].strip()       # Remove everything after the first comma
            inspector_name = inspector_name.replace("HMI", "").rstrip() # Remove "HMI" and any trailing spaces

        else:
            inspector_name = None

        # Read the PDF and extract the table on the first page
        try:
            buffer.seek(0)  # Reset the buffer position to the beginning
            tables = tabula.read_pdf(buffer, pages=1, multiple_tables=True)
        except Exception as e:
            print(f"An error occurred while reading the PDF: {e}")
            tables = []

    # Find the 'Overall effectiveness' row and extract the corresponding grade
    inspection_grade = None
    for table in tables:
        if 'Judgement' in table.columns and 'Grade' in table.columns:
            for index, row in table.iterrows():
                if row['Judgement'] == 'Overall effectiveness':

                    try:
                        inspection_grade = str(row['Grade']) # Convert the grade to a string (previously float values found in some reports)

                        if "requires improvement" in inspection_grade.lower():
                            # Some RI texts come with extra wording/baggage. Overwrite with clean val to aid clean-up
                            inspection_grade = "Requires improvement"

                    except Exception as e:
                        inspection_grade = "Unknown value type : " + inspection_grade

                        error_msg = f"unknown value found: \"unknown : {inspection_grade}\""
                        raise ValueError(error_msg)
                        
                    break
                
            if inspection_grade is not None:
                break


    # Find the inspection dates using a regular expression
    date_match = re.search(r"Inspection dates:\s*(.+)", text)

    if date_match:
        # IF there was date data


        inspection_dates = date_match.group(1).strip()
            
        # Some initial clean up based on historic data obs
        inspection_dates = inspection_dates.replace(".", "")
        inspection_dates = inspection_dates.replace("\u00A0", " ") # Remove non-breaking space (Seen in nottingham report)
        inspection_dates = re.sub(r"[\u2012\u2013\u2014\u2212\-]+", " to ", inspection_dates) # replace en dash char ("\u2013"), em dash ("\u2014"), or ("-") 
        inspection_dates = inspection_dates.split("and")[0].strip() # Need this because we have such as :
                                                                    # "8 July 2019 to 12 July 2019 and 7 August 2019 to 8 August 2019"
                                                                    # E.g. Derbyshire
        inspection_dates = re.sub(r'(\d)\s(\d)', r'\1\2', inspection_dates) # Fix white spaces between date numbers e.g. "wiltshire,	1 9 June 2019"



        if isinstance(inspection_dates, str):
            # data was as expected
            year_match = re.search(r"\d{4}", inspection_dates)
            if year_match:
                year = year_match.group(0) # get single copy of yyyy

                # Now remove the year from the inspection_dates string
                inspection_dates_cleaned = inspection_dates.replace(year, "").strip()

            else:
                # We had inspection_dates data but no recognisable year
                year = None
                inspection_dates_cleaned = inspection_dates

        else:
            # spurious data
            # inspection_dates arrived with non-str, set default val
            print("Error: inspection_dates is not a string. Type is", type(inspection_dates))
            inspection_dates_cleaned = None 


        # Now that we have already removed/cleaned those with 'and .....'
        # Split the inspection_dates_cleaned string using ' to ' as the delimiter and limit the number of splits to 1
        date_parts = inspection_dates_cleaned.split(' to ', maxsplit=1) # expect only 1 instance of 'to' between date vals
        

  
        # Get the seperate inspection date(s) 
        start_date = date_parts[0].strip()
        end_date = date_parts[1].strip() if len(date_parts) > 1 else None
        
        # Check if the month text is written in *both* the date strings
        # Required work-around as Ofsted reports contain inspection date strings in multiple formats (i/ii/iii...)
        #   i)      "15 to 26 November"  
        #   ii)     "28 February to 4 March" or "8 October to 19 October" (majority)
        #   iii)    ['8 July ', '12 July   and 7 August  to'] (*recently seen)
        #   iv)     "11 September 2017 to 5 October 2017" (double year)
        #   v)      "Inspection dates: 19 November–30 November 2018" (Bromley)
        if len(start_date) <= 2: # i.e. do we only have a date with no month text
            inspection_month = end_date.split()[1]
            start_date = f"{start_date} {inspection_month}"

        # Append the inspection year to the start_date and end_date
        start_date_full = f"{start_date} {year}"
        end_date_full = f"{end_date} {year}" if end_date else None

        # Parse the dates into datetime objects and format them as "dd/mm/yyyy"
        start_date_formatted = parser.parse(start_date_full).strftime("%d/%m/%Y")

        # Have removed date formatting on end date until bug of multiple end dates solved
        #end_date_formatted = parser.parse(end_date_full).strftime("%d/%m/%Y") if end_date_full else None

    else:
        start_date_formatted = None
        end_date_full = None


    return {'inspector_name': inspector_name, 'overall_inspection_grade': inspection_grade,'inspection_start_date': start_date_formatted,'inspection_end_date': end_date_full}


def process_provider_links(provider_links):
    """
    Processes provider links and returns a list of dictionaries containing URN, local authority, and inspection link.

    Args:
        provider_links (list): A list of BeautifulSoup Tag objects representing provider links.

    Returns:
        list: A list of dictionaries containing URN, local authority, inspection link, and, if enabled, additional inspection data.
    """
    
    data = []
    global pdf_data_capture # Bool flag
    
    for link in provider_links:
        # Extract the URN and provider name from the web link shown
        urn = link['href'].rsplit('/', 1)[-1]
        name = clean_provider_name(link.text.strip())

        # Create the provider directory path
        provider_dir = os.path.join('.', urn + '_' + name)

        # Create the provider directory if it doesn't exist
        if not os.path.exists(provider_dir):
            os.makedirs(provider_dir)

        # Get the child page content
        child_url = 'https://reports.ofsted.gov.uk' + link['href']
        child_soup = get_soup(child_url)

        # Find all publication links in the provider's child page
        pdf_links = child_soup.find_all('a', {'class': 'publication-link'})

        # Initialize a flag to indicate if an inspection link has been found
        # Important: This assumes that the provider's reports are returned/organised most recent FIRST
        found_inspection_link = False

        # Iterate through the publication links
        for pdf_link in pdf_links:

            # Check if the current/next href-link meets the selection criteria
            # This block obv relies on Ofsted continued use of nonvisual element descriptors
            # containing the type(s) of inspection text. We use  "children's services inspection"

            nonvisual_text = pdf_link.select_one('span.nonvisual').text.lower().strip()

            # For now at least, search terms hard-coded. 
            if 'children' in nonvisual_text and 'services' in nonvisual_text and 'inspection' in nonvisual_text:


                # Create the filename and download the PDF (this filetype needs to be hard-coded here)
                filename = nonvisual_text.replace(', pdf', '') + '.pdf'

                pdf_content = requests.get(pdf_link['href']).content
                with open(os.path.join(provider_dir, filename), 'wb') as f:
                    f.write(pdf_content)


               # Extract the local authority and inspection link, and add the data to the list
                if not found_inspection_link:

                    # Capture the data that will be exported about the most recent inspection only
                    local_authority = provider_dir.split('_', 1)[-1].replace('_', ' ').strip()
                    inspection_link = pdf_link['href']
                    
                    # Extract the report published date
                    report_published_date_str = filename.split('-')[-1].strip().split('.')[0] # published date appears after '-' 
            
                    report_published_date = datetime.datetime.strptime(report_published_date_str, '%d %B %Y') # str to datetime
                    report_published_date = report_published_date.strftime('%d/%m/%y') # Format dd/mm/yy
 

                    # Now get the in-document data
                    if pdf_data_capture:
                        # Opt1 : ~x4 slower runtime
                        # Only here if we have set PDF text scrape flag to True
                        # Turn this off, speeds up script if we only need the inspection documents themselves to be retrieved

                        # Scrape inside the pdf inspection reports
                        inspection_data_dict = extract_inspection_data(pdf_content)
                        
                        # Added for readability of returned data/onward
                        overall_effectiveness = inspection_data_dict['overall_inspection_grade']
                        inspector_name = inspection_data_dict['inspector_name']
                        inspection_start_date = inspection_data_dict['inspection_start_date']
                        inspection_end_date = inspection_data_dict['inspection_end_date']
                        
                        # Format the provider directory as a file path link for Excel
                        provider_dir_link = 'file:///' + provider_dir.replace("\\", "/")
                                            
                        data.append({
                                        'urn': urn,
                                        'local_authority': local_authority,
                                        'inspection_link': inspection_link,
                                        'overall_effectiveness': overall_effectiveness,
                                        'inspector_name': inspector_name,
                                        'inspection_start_date': inspection_start_date,
                                        'inspection_end_date': inspection_end_date,
                                        'publication_date': report_published_date,
                                        'local_link_to_all_inspections': provider_dir_link
                                    })
                        
                    else:
                        # Opt2 : ~x4 faster runtime
                        # Only grab the data/docs we can get direct off the Ofsted page 
                        data.append({'urn': urn, 'local_authority': local_authority, 'inspection_link': inspection_link})

                    
                    found_inspection_link = True # Flag to ensure data reporting on only the most recent inspection
    return data


def handle_pagination(soup, url_stem):
    """
    Handles pagination for a BeautifulSoup object representing a web page with paginated content.
    
    Args:
        soup (bs4.BeautifulSoup): The BeautifulSoup object representing the web page.
        url_stem (str): The base URL to which the relative path of the next page will be appended.
        
    Returns:
        str: The full URL of the next page if it exists, otherwise None.
    """
    
    # Find the pagination element in the soup object
    pagination = soup.find('ul', {'class': 'pagination'})

    # Check if the pagination element exists
    if pagination:
        # Find the next page button in the pagination element
        next_page_button = pagination.find('li', {'class': 'next'})

        # Check if the next page button exists
        if next_page_button:
            # Extract the relative URL of the next page
            next_page_url = next_page_button.find('a')['href']
            
            # Return the full URL of the next page by appending the relative URL to the base URL
            return url_stem + next_page_url

    # Return None if there is no next page button or pagination element
    return None


def save_data(data, filename, file_type='csv'):
    """
    Exports data to a specified file type.

    Args:
        data (list or dict): The data to be exported.
        filename (str): The desired name of the output file.
        file_type (str, optional): The desired file type. Defaults to 'csv'.

    Returns:
        None
    """
    if file_type == 'csv':
        filename_with_extension = filename + '.csv'
        pd.DataFrame(data).to_csv(filename_with_extension, index=False)

    elif file_type == 'excel':
        filename_with_extension = filename + '.xlsx'
        pd.DataFrame(data).to_excel(filename_with_extension, index=False)

    else:
        print(f"Error: unsupported file type '{file_type}'. Please choose 'csv' or 'xlsx'.")
        return
    
    print(f"{filename_with_extension} successfully created!")

In [10]:
#
# Scrape data

data = []
while True:
    # Fetch and parse the HTML content of the current URL
    soup = get_soup(url)
    
    # Find all 'provider' links on the page
    provider_links = soup.find_all('a', href=lambda href: href and '/provider/' in href)

    # Process the provider links and extend the data list with the results
    data.extend(process_provider_links(provider_links))

    
    # Since all results are on a single page, no need to handle pagination. 
    # Processing complete.   
    break


# Export summary data
save_data(data, export_summary_filename, export_file_type)

Got stderr: Apr 26, 2023 9:56:04 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Apr 26, 2023 9:56:04 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 26, 2023 9:56:04 AM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Apr 26, 2023 9:56:04 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Apr 26, 2023 9:56:05 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 26, 2023 9:56:21 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>

Got stderr: Apr 26, 2023 9:56:56 AM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



NameError: name 'export_file_type' is not defined